In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, dual_annealing, basinhopping
import os 
 

In [59]:
SEIR_outputs = pd.read_csv('noisy_seir_results.csv')
true_incidence = SEIR_outputs['true_incidence'].to_numpy()
symptomatic_incidence = SEIR_outputs['symptomatic_incidence'].to_numpy()

true_kernel = pd.read_csv('../../shifts/serial_interval.csv')['si']


In [112]:
def optimizer_function(real_incidence):
    convolved = np.convolve(real_incidence, true_kernel, mode='valid')

    return np.mean((convolved - symptomatic_incidence[len(true_kernel)-1:])**2) + 100000000* np.mean(np.abs(np.diff(real_incidence)/np.max(symptomatic_incidence)))
    #convolved = np.convolve(true_incidence, symptomatic_incidence) #kernel/np.sum(kernel)
    #print(convolved)
    #return np.mean((convolved - kernel[(len(symptomatic_incidence)-1):])**2)  #symptomatic_incidence


In [113]:
x0 = symptomatic_incidence
xs = minimize(optimizer_function, x0, bounds = [(0, np.inf) for _ in x0],options={'maxiter':200000})
#x0 = true_kernel['si'].to_numpy()

In [114]:
print(xs)

      fun: 857965.2533275852
 hess_inv: <302x302 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.06752850e+01,  2.13389285e+01,  2.13389285e+01,  2.13389285e+01,
        2.13389285e+01,  2.13389285e+01,  2.13389285e+01,  2.13389285e+01,
        2.13389285e+01,  2.13389285e+01,  2.13389285e+01,  2.13389285e+01,
        2.13389285e+01,  2.13389285e+01,  2.13389285e+01,  2.13389285e+01,
        2.13389285e+01,  2.13389285e+01,  2.13389285e+01,  2.13389285e+01,
        2.13389285e+01,  2.13389285e+01,  2.13389285e+01,  2.13389285e+01,
        2.13272870e+01,  2.13272870e+01,  2.13272870e+01,  2.13272870e+01,
        2.13272870e+01,  2.13272870e+01,  2.13156454e+01,  2.13156454e+01,
        2.13156454e+01,  2.13156454e+01,  2.13156454e+01,  2.13156454e+01,
        2.13156454e+01,  2.13156454e+01,  2.13156454e+01,  2.13156454e+01,
        2.13040039e+01,  2.13040039e+01,  2.13040039e+01,  2.12923624e+01,
        2.12923624e+01,  2.12923624e+01,  2.12807208e+01,  2.12807208e+01,


In [117]:
plt.figure(figsize=(20, 10))
L = (len(true_kernel)-1)
plt.plot(xs.x[L:], label='deconv')
plt.plot(true_incidence[L:], label='true')
plt.plot(symptomatic_incidence[L:], label='symptomatic')
plt.plot(np.convolve(xs.x[L:], true_kernel, mode='valid'), label='reconstruct')
plt.legend()

KeyError: 'r'